In [8]:
import torch
from torch import nn
from torch.nn import functional as F
import math
import prologue
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
#just plotting a pair
train_img, train_target, train_classes, test_img, test_target, test_classes = prologue.generate_pair_sets(1)
# plot the first training sample pair
print(train_img.shape)
fig = plt.figure
plt.imshow(train_img[0, 0], cmap='gray')
plt.show() 
plt.imshow(train_img[0, 1], cmap='gray')
plt.show() 
print(train_classes[0])

NameError: name 'prologue' is not defined

In [32]:
#the following is training two models independently, 
# one per pair of digit, and on top of that to train another one 
# that predicts the class

######################################################################

    
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32,
                               kernel_size = 4,
                               padding = (4 - 1) // 2)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 128,
                               kernel_size = 4)
                               #padding = (4 - 1) // 2)
        self.bn2 = nn.BatchNorm2d(128)
        # self.fc1 = nn.Linear( 32 , 10)

        #these are 4x4x8 + 4x4x8x32 +10x32 = 4 544 params (does tachnorm have param..?)
        
        #self.conv3 = nn.Conv2d(64, 32, kernel_size=2)
        #self.bn3 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear( 128 , 128)
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)



        
    def forward(self, x):
        #print(x.shape)
        x = F.max_pool2d(self.conv1(x), kernel_size=2)
        #print(x.shape)
        x = self.bn1(x)
        #print(x.shape)
        x = F.relu(x)
        #x = self.conv2(x)
        x = F.max_pool2d(self.conv2(x), kernel_size=2)
        x = self.bn2(x)
        #print(x.shape)
        #x = F.relu(x)
        #x = F.max_pool2d(self.conv3(x), kernel_size=2)
        #print(x.shape)
        #x = self.bn3(x)
        #x = F.relu(x)
        x = F.relu(self.fc1(x.view(-1, 128)))
        
        #x = F.relu(self.fc2(x))
        #softmax to output probabilities:
        x = self.softmax(self.fc2(x))
        
        return x
######################################################################


In [33]:
#training first model:

train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)

#isolating the first picture
first_input, first_output = train_input[:, 0:1], transform_to_one_hot(train_classes[:, 0])
mu, std = first_input.mean(), first_input.std()
first_input.sub_(mu).div_(std)

#isolating the second picture
second_input, second_output = train_input[:, 1:2], transform_to_one_hot(train_classes[:, 1])
mu, std = second_input.mean(), second_input.std()
second_input.sub_(mu).div_(std)

#making a combined dataset:
train_input = torch.cat((first_input, second_input), dim=0)
train_output = torch.cat((first_output, second_output), dim=0)
mu, std = train_input.mean(), train_input.std()

train_input.sub_(mu).div_(std)

first_model = Net()


train_model(first_model, train_input, train_output, 100, 100) #training on 1000
#train_model(first_model, first_input, first_output, 100, 100) #training on 2000




0 1.7770670726895332
10 1.22412571310997
20 0.7379602491855621
30 0.4467482455074787
40 0.2879504356533289
50 0.19443656923249364
60 0.13585059391334653
70 0.09854535688646138
80 0.07396439835429192
90 0.057191889150999486


In [34]:
first_test_input, first_test_output = test_input[:, 0:1], transform_to_one_hot(test_classes[:, 0])
#does normalizing the test input matter when already doing Batch norm?
mu, std = first_test_input.mean(), first_test_input.std()
first_test_input.sub_(mu).div_(std)

print("Train error:", compute_nb_errors(first_model, first_input, first_output, 20)/10)
#first_model(first_input[0:1])
print("Test error:", compute_nb_errors(first_model, first_test_input, first_test_output, 20)/10)


second_test_input, second_test_output = test_input[:, 1:2], transform_to_one_hot(test_classes[:, 1])
#does normalizing the test input matter when already doing Batch norm?
mu, std = second_test_input.mean(), second_test_input.std()
second_test_input.sub_(mu).div_(std)

print('done :),')

#NOTE: the test results are highly irregular

Train error: 1.1
Test error: 6.1
done :),


In [38]:
# The following is meant to be combined with the above network to first predict classes, 
# and then predict which one is bigger. Feed it the two prob dists
class geqNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(20 , 2)
        self.sig1 = nn.Sigmoid()
        #self.fc2 = nn.Linear(40, 2)
        #self.sig2 = nn.Sigmoid()



    def forward(self, x):
        #print(x.shape)
        x = self.fc1(x.view(-1, 20))
        x = self.sig1(x)
        #x = self.fc2(x)
        #x = self.sig2(x)
        
        #print(x.shape)
        #x = self.sm(x)
        return x
    
######################################################################


In [39]:
geq = geqNet()

train_input = torch.detach(torch.cat((first_model(first_input), first_model(second_input)), dim=1))
mu, std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)

train_output = torch.unsqueeze(train_target, 1)#.to(torch.float32)
train_output_new = torch.zeros((len(train_output), 2))
for i in range(len(train_output)):
    train_output_new[i, train_output[i]]=1

train_model(geq, train_input, train_output_new, 100, 100, criterion=torch.nn.BCEWithLogitsLoss())




0 7.24994432926178
10 6.781591057777405
20 6.465334713459015
30 6.263695120811462
40 6.128804266452789
50 6.0326685309410095
60 5.960366070270538
70 5.903654098510742
80 5.8577030301094055
90 5.819515824317932


In [47]:
#compute nb errors:
output = geq(torch.detach(torch.cat((first_model(first_test_input), first_model(second_test_input)), dim=1)))
nb=0
for i in range(1000):
    if output[i, 0]>output[i, 1]:
        if test_target[i] == 1:
            nb+=1
    else:
        if test_target[i] == 0: #
            nb+=1
print('nb of errors:', nb)
            

nb of errors: 159


In [52]:
#Testing is the hand crafted expectation works. It does not, is this a consequence of lack of idenpendence?  
#Observation: 3 and 8 have a very very bad blablabla
#####################################################
def average(prob_vect):
    avg=0
    for i in range(10):
        avg = i*prob_vect[i]
    return avg
#####################################################
print(first_model(first_test_input[2:4]), first_model(second_test_input[2:4]))
print(test_classes[2:4])
print(test_target[2:4])

#Hand made layer:
output1 = first_model(first_test_input)
output2 = first_model(second_test_input)

nb=0
for i in range(len(test_target)):
    number1 = average(output1[i])
    number2 = average(output2[i])
    if number1-number2 > 0:
        if test_target[i] == 1:
            nb+=1
    else:
        if test_target[i] == 0:
            nb+=1
                
print('nb of errors:', nb)



tensor([[0.0090, 0.0049, 0.0119, 0.0046, 0.0176, 0.0114, 0.0363, 0.0029, 0.8549,
         0.0465],
        [0.0351, 0.0282, 0.0511, 0.2064, 0.0085, 0.0235, 0.0057, 0.6307, 0.0045,
         0.0062]], grad_fn=<SoftmaxBackward>) tensor([[0.0037, 0.0045, 0.0223, 0.0160, 0.0022, 0.0227, 0.0550, 0.0013, 0.8655,
         0.0068],
        [0.0074, 0.0252, 0.0049, 0.0058, 0.0107, 0.0028, 0.0018, 0.9330, 0.0019,
         0.0064]], grad_fn=<SoftmaxBackward>)
tensor([[8, 8],
        [3, 7]])
tensor([1, 1])
nb of errors: 327


In [6]:
#These are helper functions, should be in separate file
######################################################################
def transform_to_one_hot(train): #train is a 1 dimensional tensor and transorms to 2 dimensional
    one_hot = torch.zeros((len(train), 10))
    for i in range(len(train)):
        one_hot[i, train[i]] = 1
    return one_hot


def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 100, criterion=nn.MSELoss()):
    eta = 1e-1

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()

            model.zero_grad()
            loss.backward()

            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad
        if e%10 == 0:
            print(e, acc_loss)

def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        #print(predicted_classes)
        for k in range(mini_batch_size):
            if target[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

######################################################################



In [28]:
#!/usr/bin/env python

# Any copyright is dedicated to the Public Domain.
# https://creativecommons.org/publicdomain/zero/1.0/

# Written by Francois Fleuret <francois@fleuret.org>

import torch
from torch import nn
from torch.nn import functional as F

import prologue




def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors



In [ ]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(500)
######################################################################

mini_batch_size = 100

######################################################################
### Question 2: Number of testerrors

for k in range(10):
    model = Net(100)
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))


In [13]:
######################################################################

#Weight sharing attempt. Siamese network

######################################################################

    
class SiameseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32,
                               kernel_size = 4,
                               padding = (4 - 1) // 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 128,
                               kernel_size = 4)
                               #padding = (4 - 1) // 2)
        self.bn2 = nn.BatchNorm2d(128)
        # self.fc1 = nn.Linear( 32 , 10)

        #these are 4x4x8 + 4x4x8x32 +10x32 = 4 544 params (does tachnorm have param..?)
        
        #self.conv3 = nn.Conv2d(64, 32, kernel_size=2)
        #self.bn3 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear( 128 , 128)
        self.fc2 = nn.Linear(128, 10)
        
        self.softmax = nn.Softmax(dim=1) 
        #self.finalLin = nn.Linear(20 , 2)




        
    def forward(self, x):
        #print(x.shape)
        x = torch.stack((F.max_pool2d(self.conv1(x[:, 0:1]), kernel_size=2), 
                        F.max_pool2d(self.conv1(x[:, 1:2]), kernel_size=2)), dim=1)
        #print(x.shape)
        x = torch.stack((self.bn1(x[:, 0]), self.bn1(x[:, 1])), dim=1)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)
        #x = self.conv2(x)
        x = torch.stack((F.max_pool2d(self.conv2(x[:, 0]), kernel_size=2), 
                        F.max_pool2d(self.conv2(x[:, 1]), kernel_size=2)), dim=1)
        x = torch.stack((self.bn2(x[:, 0]), self.bn2(x[:, 1])), dim=1)
        #print(x.shape)
        #x = F.relu(x)
        #x = F.max_pool2d(self.conv3(x), kernel_size=2)
        #print(x.shape)
        #x = self.bn3(x)
        #x = F.relu(x)
        x = torch.stack((F.relu(self.fc1(x[:, 0].view(-1, 128))), F.relu(self.fc1(x[:, 1].view(-1, 128)))), dim=1)
        x = torch.stack((self.softmax(self.fc2(x[:, 0].view(-1, 128))), self.softmax(self.fc2(x[:, 1].view(-1, 128)))), dim=1)
        
        #x = self.sig1(x)
        #print(x.shape)
        #print(x.view(-1, 20).shape)
        #x = self.finalLin(x.view(-1, 20)) #batch size!
        
        
        
        return x
######################################################################


def big_transform_to_one_hot(train): #train is a??
    one_hot = torch.zeros((len(train),2, 10))
    for i in range(len(train)):
        one_hot[i, 0, train[i, 0]] = 1
        one_hot[i, 1, train[i, 1]] = 1
    return one_hot

In [14]:
#training first model:

train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)

mu, std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)

first_model = SiameseNet()

train_model(first_model, train_input, big_transform_to_one_hot(train_classes), 100, 100)
#train_model(first_model, second_input, second_output, 100, 100)



0 0.8988799825310707
10 0.773406982421875
20 0.6200904995203018
30 0.47546733543276787
40 0.3675930052995682
50 0.2942831963300705
60 0.2282073125243187
70 0.17376418318599463
80 0.13541717734187841
90 0.10892943385988474


In [16]:

mu, std = test_input.mean(), test_input.std()
test_input.sub_(mu).div_(std)

print("Train error:", compute_nb_errors(first_model, train_input, big_transform_to_one_hot(train_classes), 20)/10)
#first_model(first_input[0:1])
print("Test error:", compute_nb_errors(first_model, test_input, big_transform_to_one_hot(test_classes), 20)/10)



print('it worked!! :)')

Train error: 7.4
Test error: 15.1
it worked!! :)


In [17]:
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 100, criterion=nn.MSELoss()):
    eta = 1e-1

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()

            model.zero_grad()
            loss.backward()

            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad
        if e%10 == 0:
            print(e, acc_loss)
            
def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        output1, output2 = output[:, 0], output[:, 1]
        _, predicted_classes1 =  output1.max(1)
        _, predicted_classes2 =  output2.max(1)
        #print(predicted_classes)
        for k in range(mini_batch_size):
            if target[b + k, 0, predicted_classes1[k]] <= 0 or target[b + k, 1, predicted_classes2[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

######################################################################

In [18]:
from SiameseNet import *

In [19]:
Net = SiameseNet()

In [20]:
Net.train_model(train_input, big_transform_to_one_hot(train_classes), 100, 100)

0 0.8938411623239517
10 0.7547252401709557
20 0.627328559756279
30 0.4917127452790737
40 0.3789517432451248
50 0.2846033647656441
60 0.212232930585742
70 0.160043403506279
80 0.12555128056555986
90 0.10193012375384569


In [22]:
Net.compute_nb_errors(test_input, big_transform_to_one_hot(test_classes), 25)/10

12.6